<a href="https://colab.research.google.com/github/baiyinnamula/EasyToUnderstandPandas/blob/main/%E7%AC%AC6%E7%AB%A0_Pandas%E5%88%86%E7%BB%84%E8%81%9A%E5%90%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_excel('https://www.gairuo.com/file/data/dataset/team.xlsx')

## 6.1 概述


### 6.1.1 原理
新西兰统计学家、数据科学家哈德利·威克姆（Hadley Wickham，
众多热门R语言包的作者）在其知名论文“The Split-Apply-Combine
Strategy for Data Analysis”
[1]中阐述了“拆分－应用－合并”（Split-ApplyCombine）策略在数据分析中的应用，Pandas按照这个思路给出了最佳
实践。

### 6.1.2 groupby 语法

### 6.1.3 DataFrame应用分组

In [ ]:
df

In [ ]:
df.groupby('team').sum()

In [ ]:
df.groupby('team').agg({
    'Q1':sum,
    'Q2':'count',
    'Q3': 'mean',
    'Q4': max
})

In [ ]:
df.groupby('team').agg({
    'Q1':[sum, 'std', max],
    'Q2':'count',
    'Q3': 'mean',
    'Q4': max
})

### 6.1.4 Series应用分组

In [ ]:
df.Q1.groupby(df.team).sum()

## 6.2 分组

### 6.2.1 分组对象

In [ ]:
df.groupby('team')

In [ ]:
df.Q1.groupby(df.team)

### 6.2.2 按标签分组

In [ ]:
grouped = df.groupby('team')
grouped.get_group('D')

### 6.2.3 表达式

In [ ]:
df.groupby(lambda x: x%2 == 0).sum()

In [ ]:
df.groupby(df.index%2 == 0).sum()

In [ ]:
df.groupby(lambda x: x >= 50).sum()

In [ ]:
df.groupby(df.index >= 50).sum()

In [ ]:
df.groupby(lambda x : 'Q' in x, axis=1).sum()

In [ ]:
df

In [ ]:
df.groupby(df.name.str[0]).count()

In [ ]:
df.groupby(df.team.isin(['A', 'B'])).count()

### 6.2.4 函数分组

In [ ]:
# 按姓名首字母为元音、辅音分组
def get_letter_type(letter):
  if letter[0].lower() in 'aeiou':
    return '元音'
  else:
    return '辅音'


df.set_index('name').groupby(get_letter_type).sum()

### 6.2.5 多种方法混合

In [ ]:
# 按team、姓名首字母是否为元音分组
df.groupby(['team', df.name.apply(get_letter_type)]).sum()

### 6.2.6 用pipe调用分组方法

In [ ]:
df.pipe(pd.DataFrame.groupby, 'team').sum()

### 6.2.7 分组器Grouper

In [ ]:
df.groupby(pd.Grouper('team')).sum()

### 6.2.8 索引

In [ ]:
df.groupby('team', as_index=False).sum()

In [ ]:
df.groupby('team', as_index=True).sum()

### 6.2.9 排序

In [ ]:
df.groupby('team', sort=False).sum()

## 6.3 分组对象操作

In [ ]:
# 分组，为了方便案例介绍，删去name列，分组后全为数字
grouped = df.drop('name', axis=1).groupby('team')
grouped

In [ ]:
grouped.sum()

In [ ]:
grouped.groups

### 6.3.1 选择分组

In [ ]:
grouped.groups.keys()

In [ ]:
# 用团队和姓名首字母分组
grouped2 = df.groupby(['team', df.name.str[0]])
# 选择B组、姓名以A开头的数据
grouped2.get_group(('B', 'A'))

In [ ]:
grouped.indices

### 6.3.2 迭代分组

In [ ]:
for name, group  in grouped:
  print(type(name))
  print(type(group))

### 6.3.3 选择列

In [ ]:
grouped[['Q1', 'Q2']]

### 6.3.4 应用函数apply()

In [ ]:
df.groupby('team').apply(lambda x: x*2)

In [ ]:
df.groupby('team').apply(lambda x: x['name'].to_list())

In [ ]:
df.groupby('team').apply(lambda x: x['name'].to_list()).A

In [ ]:
# 各组Q1（为参数）成绩最高的前三个
def first_3(df_, c):
  return df_[c].sort_values(ascending=False).head(3)

In [ ]:
df.set_index('name').groupby('team').apply(first_3, 'Q1')

In [ ]:
# 通过设置group_keys，可以使分组字段不作为索引
df.set_index('name').groupby('team', group_keys=False).apply(first_3, 'Q1')

In [ ]:
(
df.groupby('team').apply(lambda x: pd.Series({
  'Q1_sum' : x['Q1'].sum(),
  'Q1_max' : x['Q1'].max(),
  'Q2_mean' : x['Q2'].mean(),
  'Q4_prodsum' : (x['Q4'] * x['Q4']).sum()
}))
)
# 定义一个函数
def f_mi(x):
  d = []
  d.append(x['Q1'].sum())
  d.append(x['Q2'].max())
  d.append(x['Q3'].mean())
  d.append((x['Q4'] * x['Q4']).sum())
  return pd.Series(d, index=[['Q1', 'Q2', 'Q3', 'Q4'], ['sum', 'max', 'mean', 'prodsum']])
# 使用函数
df.groupby('team').apply(f_mi)

In [ ]:
# 定义了A组和B组平均值的差值
def mean_diff(x):
  return x.get_group('A').mean() - x.get_group('B').mean()


df.groupby('team').pipe(mean_diff)

### 6.3.5 管道方法pipe()

In [ ]:
df.drop('name', axis=1).groupby('team').pipe(lambda x: x.max() - x.min())

### 6.3.6 转换方法transform()

In [ ]:
df.groupby('team').transform(np.mean)

In [ ]:
df.groupby('team').transform(max)

In [ ]:
df.groupby('team').transform(np.std)

In [ ]:
## 使用函数，和上一个学生的差值（没有处理姓名列）
df.groupby('team').transform(lambda x: x.shift(-1))

In [ ]:
def score(gb):
  return (gb - gb.mean()) / gb.std() * 10
grouped.transform(score)

In [ ]:
#Q1成绩大于60的组的所有成员
df[df.groupby('team').transform('mean').Q1 > 60]

### 6.3.7 筛选方法filter()

In [ ]:
# 每组每个季度的平均分
df.groupby('team').mean()

In [ ]:
# 每组4个季度的平均分的平均分为本组的总平均分
df.groupby('team').mean().mean(1)

In [ ]:
# 筛选出所在组总平均分大于51的成员
df.groupby('team').filter(lambda x: x.mean(1).mean() > 51)

In [ ]:
# Q1成绩至少有一个大于97的组
df.groupby('team').filter(lambda x: (x['Q1'] > 97).any())

In [ ]:
# # 所有成员平均成绩大于60的组
df.groupby('team').filter(lambda x: (x.mean() >= 60).all())

In [ ]:
# Q1所有成员成绩之和超过1060的组
df.groupby('team').filter(lambda g: g.Q1.sum() > 1060)

### 6.3.8 其他功能

In [ ]:
df.groupby('team').first() # 组内第一个
df.groupby('team').last() # 组内最后一个
df.groupby('team').ngroups # 5（分组数）
df.groupby('team').ngroup() # 分组序号
grouped.backfill()
grouped.bfill()
df.groupby('team').head() # 每组显示前5个
grouped.tail(1) # 每组最后一个
grouped.rank() # 排序值
grouped.fillna(0)
# grouped.indices() # 组名:索引序列组成的字典

In [ ]:
# 分组中的第几个值
gp.nth(1) # 第一个
gp.nth(-1) # 最后一个
gp.nth([-2, -1])
# 第n个非空项
gp.nth(0, dropna='all')
gp.nth(0, dropna='any')
df.groupby('team').shift(-1) # 组内移动
grouped.tshift(1) # 按时间周期移动
df.groupby('team').any()
df.groupby('team').all()
df.groupby('team').rank() # 在组内的排名
# 仅 SeriesGroupBy 可用
df.groupby("team").Q1.nlargest(2) # 每组最大的两个
df.groupby("team").Q1.nsmallest(2) # 每组最小的两个
df.groupby("team").Q1.nunique() # 每组去重数量
df.groupby("team").Q1.unique() # 每组去重值
df.groupby("team").Q1.value_counts() # 每组去重值及数量
df.groupby("team").Q1.is_monotonic_increasing # 每组值是否单调递增
df.groupby("team").Q1.is_monotonic_decreasing # 每组值是否单调递减
# 仅 DataFrameGroupBy 可用
df.groupby("team").corrwith(df2) # 相关性

### 6.3.9 小结

本节介绍了对分组的基本操作和一些函数方法，特别要注意分辨以
下三个方法。
* apply()：最为灵活的处理方法，可以对数据完成操作后返回各种
形式的数据。
* transform()：对数据处理完后返回原型形状的数据，可以类比为
对一个汽车不改变结构，只重新进行涂装。
* filter()：每个分组传入后，通过计算返回这个分组的真假值，所
有为真的留下，作为最终的结果。

其中transform()和filter()计算的都是每个分组的整体结果。

##6.4 聚合统计

### 6.4.1 描述统计

In [ ]:
df.groupby('team').describe()

In [ ]:
df.groupby('team').describe().T

### 6.4.2 统计函数

In [ ]:
# 各组平均值
grouped.mean()

In [ ]:
df.groupby('team').describe() # 描述性统计


In [ ]:
df.groupby('team').sum() # 求和
df.groupby('team').count() # 每组数量，不包括缺失值
df.groupby('team').max() # 求最大值
df.groupby('team').min() # 求最小值
df.groupby('team').size() # 分组数量
df.groupby('team').mean() # 平均值
df.groupby('team').median() # 中位数


In [ ]:
df.groupby('team').std() # 标准差


In [ ]:
df.groupby('team').var() # 方差


In [ ]:
grouped.corr() # 相关性系数


In [ ]:
grouped.sem() # 标准误差


In [ ]:
grouped.prod() # 乘积


In [ ]:
grouped.cummax() # 每组的累计最大值
grouped.cumsum() # 累加
grouped.mad() # 平均绝对偏差

### 6.4.3 聚合方法agg()

In [ ]:
# 所有列使用一个计算方法
df.groupby('team').agg(sum)

In [ ]:
df.groupby('team').agg(np.size)

In [ ]:
grouped['Q1'].agg(np.mean)

In [ ]:
# 每个字段使用多个计算方法
grouped[['Q1','Q3']].agg([np.sum, np.mean, np.std])

In [ ]:
df.groupby('team').agg({'Q1': ['min', 'max'], 'Q2': 'sum'})

In [ ]:
df.groupby('team').Q1.agg(Mean='mean', Sum='sum')


In [ ]:
df.groupby('team').agg(Mean=('Q1', 'mean'), Sum=('Q2', 'sum'))


In [ ]:
df.groupby('team').agg(
  Q1_max=pd.NamedAgg(column='Q1', aggfunc='max'),
  Q2_min=pd.NamedAgg(column='Q2', aggfunc='min')
)

In [ ]:
df.groupby('team').agg(**{
'1_max':pd.NamedAgg(column='Q1', aggfunc='max')})

In [ ]:
# 聚合结果使用函数
# lambda/函数，所有方法都可以用
def max_min(x):
  return x.max() - x.min()
# 定义函数
df.groupby('team').Q1.agg(Mean='mean',
  Sum='sum',
  Diff=lambda x: x.max() - x.min(),
  Max_min=max_min
)

In [ ]:
df.groupby('team').agg(max_min)

### 6.4.4 时序重采样方法resample()

In [ ]:
idx = pd.date_range('1/1/2020', periods=100, freq='T')
df2 = pd.DataFrame(data={'a':[0, 1]*50, 'b':1}, index=idx)
df2

In [ ]:
df2.groupby('a').resample('20T').sum()

In [ ]:
df2.groupby('a').resample('3T').sum()

In [ ]:
df2.groupby('a').resample('30S').sum()

In [ ]:
df2.groupby('a').resample('M').sum()

In [ ]:
# 以右边时间点为标识
df2.groupby('a').resample('3T', closed='right').sum()

### 6.4.5 组内头尾值

In [ ]:
df.groupby('team').first()

In [ ]:
df.groupby('team').last()

### 6.4.6 组内分位数

In [ ]:
# 二分位数，即中位数
df.groupby('team').median() # 同下


In [ ]:
df.groupby('team').quantile()


In [ ]:
df.groupby('team').quantile(0.5)

常用的还有三分位quantile(0.33)、四分位quantile(0.25)等。


### 6.4.7 组内差值

In [ ]:
df.drop('name', axis=1).groupby('team').diff()

## 6.5 数据分箱


Pandas主要基于以两个函数实现连续数据的离散化处理。
* pandas.cut：根据指定分界点对连续数据进行分箱处理。
* pandas.qcut：根据指定区间数量对连续数据进行等宽分箱处理。
所谓等宽，指的是每个区间中的数据量是相同的。

### 6.5.1 定界分箱pd.cut()

In [ ]:
pd.cut(df.Q1, bins=[0, 60, 100])

In [ ]:
df.Q1.groupby(pd.cut(df.Q1, bins=[0, 60, 100])).count()

In [ ]:
df.groupby(pd.cut(df.Q1, bins=[0, 60, 100])).count()

In [ ]:
# 不显示区间，使用数字作为每个箱子的标签，形式如0，1，2，n等
pd.cut(df.Q1, bins=[0, 60, 100],labels=False)

In [ ]:
# 指定标签名
pd.cut(df.Q1, bins=[0, 60, 100],labels=['不及格','及格',])

In [ ]:
# 包含最低部分
pd.cut(df.Q1, bins=[0, 60, 100], include_lowest=True)

In [ ]:
# 是否为右闭区间，下例为[89, 100)
pd.cut(df.Q1, bins=[0, 89, 100], right=False)

### 6.5.2 等宽分箱pd.qcut()

In [ ]:
pd.qcut(df.Q1, q=2)

In [ ]:
pd.qcut(df.Q1, q=2).unique()

In [ ]:
df.Q1.groupby(pd.qcut(df.Q1, q=2)).count()

In [ ]:
df.groupby(pd.qcut(df.Q1, q=3)).max()

## 6.6 分组可视化


### 6.6.1 绘图方法plot()

In [ ]:
grouped = df.set_index('name').groupby('team')

In [ ]:
grouped.plot()

还可以通过plot.x()或者plot(kind='x')的形式调用其他形状的图形，
比如：
* plot.line：折线图
* plot.pie：饼图
* plot.bar：柱状图
* plot.hist：直方图
* plot.box：箱形图
* plot.area：面积图
* plot.scatter：散点图
* plot.hexbin：六边形分箱图


### 6.6.2 直方图hist()

In [ ]:
grouped.hist()

In [ ]:
grouped.boxplot(figsize=(15, 12))

In [ ]:
df.boxplot(by='team', figsize=(15, 10))

## 6.7 本章小结

本章全面介绍了分组聚合的数据操作原理，依次可以分为以下部分。
* 分拆（split）：将DataFrame或Series按照一定的规则进行分组，
生成分组对象，分组对象中包含多个子DataFrame或Series。
* 应用（apply）：对每个组进行操作或数据统计，如算平均数
据、求方差、取中位数，还可以使用函数进行复杂的操作和计
算。
* 合并（combine）：将每组的计算结果再拼合起来，最终得到一
个DataFrame或Series，或者直接进行可视化显现。

数据的分组聚合是数据分析的常规手段，旨在将有共性的事物进行
分组统计，最终对各组进行比较，从而发现规律。希望大家在使用中能
够准确掌握，灵活运用。